In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
if sys.version_info[0] == 2:
    import xml.etree.cElementTree as ET
else:
    import xml.etree.ElementTree as ET


In [2]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

In [3]:
parser = argparse.ArgumentParser(
    description='Single Shot MultiBox Detector Training With Pytorch')
train_set = parser.add_mutually_exclusive_group()
parser.add_argument('--foo', action='store_true')
parser.add_argument('--dataset', default='VOC', choices=['VOC', 'COCO'],
                    type=str, help='VOC or COCO')
parser.add_argument('--dataset_root', default=VOC_ROOT,
                    help='Dataset root directory path')
parser.add_argument('--basenet', default='vgg16_reducedfc.pth',
                    help='Pretrained base model')
parser.add_argument('--batch_size', default=4, type=int,
                    help='Batch size for training')
parser.add_argument('--resume', default=None, type=str,
                    help='Checkpoint state_dict file to resume training from')
parser.add_argument('--start_iter', default=0, type=int,
                    help='Resume training at this iter')
parser.add_argument('--num_workers', default=4, type=int,
                    help='Number of workers used in dataloading')
parser.add_argument('--cuda', default=True, type=str2bool,
                    help='Use CUDA to train model')
parser.add_argument('--lr', '--learning-rate', default=1e-4, type=float,
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float,
                    help='Momentum value for optim')
parser.add_argument('--weight_decay', default=5e-4, type=float,
                    help='Weight decay for SGD')
parser.add_argument('--gamma', default=0.1, type=float,
                    help='Gamma update for SGD')
parser.add_argument('--visdom', default=False, type=str2bool,
                    help='Use visdom for loss visualization')
parser.add_argument('--save_folder', default='weights/',
                    help='Directory for saving checkpoint models')

_StoreAction(option_strings=['--save_folder'], dest='save_folder', nargs=None, const=None, default='weights/', type=None, choices=None, help='Directory for saving checkpoint models', metavar=None)

In [4]:
import sys
sys.argv=['']
del sys

In [5]:
args = parser.parse_args()

In [6]:
if torch.cuda.is_available():
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not args.cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

if not os.path.exists(args.save_folder):
    os.mkdir(args.save_folder)

In [7]:
cfg=voc

In [8]:
cfg

{'num_classes': 3,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [9]:
dataset = VOCDetection(root=args.dataset_root,
                               transform=SSDAugmentation(cfg['min_dim'],
                                                         MEANS))

In [10]:
dataset.name

'VOC0712'

In [11]:
len(dataset)

1423

In [12]:
img_id=dataset.ids[1]

In [13]:
import pandas as pd

In [14]:
anno = pd.read_csv(dataset._annopath % img_id)

In [15]:
anno['xmin']=anno['x']
anno['ymin']=anno['y']
anno['xmax']=anno['x']+anno['dx']
anno['ymax']=anno['y']+anno['dy']

In [16]:
anno

,#item,x,y,dx,dy,label,xmin,ymin,xmax,ymax
0,0,341.428571,287.142857,56.026203,58.522639,mango,341.428571,287.142857,397.454774,345.665496
1,1,438.095238,309.523810,56.026203,70.812393,mango,438.095238,309.523810,494.121441,380.336202
2,2,421.904762,373.333333,50.423582,63.731153,mango,421.904762,373.333333,472.328344,437.064487
3,3,452.380952,442.380952,44.033356,55.654447,mango,452.380952,442.380952,496.414309,498.035399


In [17]:
cols=np.append(anno.columns[6:],anno.columns[5])

In [18]:
df=anno[cols]

In [19]:
df.loc[:,'xmin']/=500

/home/yuvraj/anaconda3/envs/tf/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
class_to_ind= dict(zip(VOC_CLASSES, range(len(VOC_CLASSES))))

In [21]:
class_to_ind

{'mango': 0, 'almond': 1}

In [22]:
df.loc[:,'label']=class_to_ind[df['label'][0]]

In [23]:
df

,xmin,ymin,xmax,ymax,label
0,0.682857,287.142857,397.454774,345.665496,0
1,0.876190,309.523810,494.121441,380.336202,0
2,0.843810,373.333333,472.328344,437.064487,0
3,0.904762,442.380952,496.414309,498.035399,0


In [24]:
res=df.values

In [25]:
res

array([[  0.68285714, 287.14285714, 397.45477398, 345.66549573,
          0.        ],
       [  0.87619048, 309.52380952, 494.12144064, 380.33620221,
          0.        ],
       [  0.84380952, 373.33333333, 472.3283442 , 437.06448675,
          0.        ],
       [  0.9047619 , 442.38095238, 496.41430871, 498.03539937,
          0.        ]])

In [26]:
trans=VOCAnnotationTransform()

In [27]:
img=cv2.imread('000005.jpg')

In [28]:
height,width,channels=img.shape

In [29]:
import pandas as pd
target=pd.read_csv('/home/yuvraj/Desktop/work/HM/acfr-fruit-dataset/mangoes/annotations/20151124T024327.193809_i1590j799.csv')

In [30]:
target

,#item,x,y,dx,dy,label
0,0,273.000000,344.000000,47.158954,47.158954,mango
1,1,248.000000,356.000000,47.158954,47.158954,mango
2,2,373.000000,395.000000,47.158954,47.158954,mango
3,3,391.000000,418.000000,47.158954,47.158954,mango
4,4,332.631579,381.578947,32.000000,42.592000,mango
5,5,327.368421,267.368421,28.512000,31.049568,mango
6,6,344.736842,242.105263,31.363200,41.326975,mango


In [31]:
target=trans(target,height,width)

In [32]:
target

array([[0.728     , 0.91733333, 0.64031791, 0.78231791, 0.        ],
       [0.66133333, 0.94933333, 0.59031791, 0.80631791, 0.        ],
       [0.99466667, 1.05333333, 0.84031791, 0.88431791, 0.        ],
       [1.04266667, 1.11466667, 0.87631791, 0.93031791, 0.        ],
       [0.88701754, 1.01754386, 0.72926316, 0.84834189, 0.        ],
       [0.87298246, 0.71298246, 0.71176084, 0.59683598, 0.        ],
       [0.91929825, 0.64561404, 0.75220008, 0.56686448, 0.        ]])

In [33]:
target=np.array(target)

In [34]:
target[:, :4]

array([[0.728     , 0.91733333, 0.64031791, 0.78231791],
       [0.66133333, 0.94933333, 0.59031791, 0.80631791],
       [0.99466667, 1.05333333, 0.84031791, 0.88431791],
       [1.04266667, 1.11466667, 0.87631791, 0.93031791],
       [0.88701754, 1.01754386, 0.72926316, 0.84834189],
       [0.87298246, 0.71298246, 0.71176084, 0.59683598],
       [0.91929825, 0.64561404, 0.75220008, 0.56686448]])

In [35]:
target[:, 4]

array([0., 0., 0., 0., 0., 0., 0.])

In [36]:
meme=SSDAugmentation(cfg['min_dim'],MEANS)

In [37]:
img, boxes, labels = meme(img, target[:, :4], target[:, 4])

In [38]:
img

array([[[-68.364365 , -65.83083  , -93.760475 ],
        [-62.2554   , -53.612892 , -81.542534 ],
        [-39.20639  , -18.273087 , -43.96884  ],
        ...,
        [-21.676765 ,  48.864716 , -34.709396 ],
        [-25.262245 ,  45.044357 , -38.25618  ],
        [ -8.971558 ,  52.17154  , -20.456436 ]],

       [[-67.69603  , -66.49916  , -94.4288   ],
        [-58.654762 , -52.000534 , -79.93018  ],
        [-34.890076 , -15.641533 , -41.571754 ],
        ...,
        [-24.141289 ,  47.444458 , -37.591866 ],
        [-23.065063 ,  49.279938 , -37.18741  ],
        [-12.31321  ,  52.17154  , -25.647942 ]],

       [[-66.55429  , -67.6409   , -95.57054  ],
        [-52.503666 , -49.24611  , -77.17575  ],
        [-27.51638  , -11.145958 , -37.47673  ],
        ...,
        [-28.351517 ,  45.018173 , -42.516098 ],
        [-19.311554 ,  56.515717 , -35.361588 ],
        [-18.021873 ,  52.17154  , -34.516754 ]],

       ...,

       [[ 40.387558 , -20.509583 ,  -8.558174 ],
        [ 4

In [39]:
boxes

array([[0.58929168, 0.98170732, 0.44067797, 0.67298302],
       [0.67403744, 1.05487805, 0.55367232, 0.72786107],
       [0.46820196, 0.51444159, 0.19494499, 0.24886275],
       [0.39966087, 0.36039795, 0.11644365, 0.18033036]])

In [40]:
labels

array([0., 0., 0., 0.])

In [41]:
class_to_ind=dict(zip(VOC_CLASSES, range(len(VOC_CLASSES))))

In [42]:
class_to_ind

{'mango': 0, 'almond': 1}

res=[]
for obj in target.iter('object'):
    difficult = int(obj.find('difficult').text) == 1
    name = obj.find('name').text.lower().strip()
    bbox = obj.find('bndbox')

    pts = ['xmin', 'ymin', 'xmax', 'ymax']
    bndbox = []
    for i, pt in enumerate(pts):
        cur_pt = int(bbox.find(pt).text) - 1
        # scale height or width
        cur_pt = cur_pt / width if i % 2 == 0 else cur_pt / height
        bndbox.append(cur_pt)
    label_idx = class_to_ind[name]
    bndbox.append(label_idx)
    res += [bndbox]  # [xmin, ymin, xmax, ymax, label_ind]
    # img_id = target.find('filename').text[:-4]


for obj in target.iter('object'):
    print(obj)

res

In [43]:
import pandas as pd
annotation=pd.read_csv('/home/yuvraj/Desktop/work/HM/acfr-fruit-dataset/mangoes/annotations/20151124T024327.193809_i1590j799.csv')

In [44]:
annotation

,#item,x,y,dx,dy,label
0,0,273.000000,344.000000,47.158954,47.158954,mango
1,1,248.000000,356.000000,47.158954,47.158954,mango
2,2,373.000000,395.000000,47.158954,47.158954,mango
3,3,391.000000,418.000000,47.158954,47.158954,mango
4,4,332.631579,381.578947,32.000000,42.592000,mango
5,5,327.368421,267.368421,28.512000,31.049568,mango
6,6,344.736842,242.105263,31.363200,41.326975,mango


In [45]:
height

375

In [46]:
width

500

In [47]:
channels

3

In [48]:
img2=cv2.imread('/home/yuvraj/Desktop/work/HM/acfr-fruit-dataset/mangoes/images/20151124T024327.193809_i1590j799.png')

In [49]:
heght2,width2,channels2=img2.shape

In [50]:
heght2

500

In [51]:
width2

500

In [52]:
channels2

3

In [53]:
res

array([[  0.68285714, 287.14285714, 397.45477398, 345.66549573,
          0.        ],
       [  0.87619048, 309.52380952, 494.12144064, 380.33620221,
          0.        ],
       [  0.84380952, 373.33333333, 472.3283442 , 437.06448675,
          0.        ],
       [  0.9047619 , 442.38095238, 496.41430871, 498.03539937,
          0.        ]])

res=[]
for obj in target.iter('object'):
    difficult = int(obj.find('difficult').text) == 1
    name = obj.find('name').text.lower().strip()
    bbox = obj.find('bndbox')

    pts = ['xmin', 'ymin', 'xmax', 'ymax']
    bndbox = []
    for i, pt in enumerate(pts):
        cur_pt = int(bbox.find(pt).text) - 1
        bndbox.append(cur_pt)
    label_idx = class_to_ind[name]
    bndbox.append(label_idx)
    res += [bndbox]  # [xmin, ymin, xmax, ymax, label_ind]
    # img_id = target.find('filename').text[:-4]


In [54]:
res

array([[  0.68285714, 287.14285714, 397.45477398, 345.66549573,
          0.        ],
       [  0.87619048, 309.52380952, 494.12144064, 380.33620221,
          0.        ],
       [  0.84380952, 373.33333333, 472.3283442 , 437.06448675,
          0.        ],
       [  0.9047619 , 442.38095238, 496.41430871, 498.03539937,
          0.        ]])

In [55]:
cv2.imshow('image',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [56]:
int(annotation['x'][5])

327

In [57]:
cv2.rectangle(img2,(int(annotation['x'][0]),int(annotation['y'][0])),(int(annotation['x'][0]+annotation['dx'][0]),int(annotation['y'][0]+annotation['dy'][0])),(0,255,0),1)

array([[[82, 66, 78],
        [84, 66, 80],
        [86, 63, 79],
        ...,
        [96, 75, 82],
        [95, 75, 85],
        [96, 75, 89]],

       [[80, 64, 77],
        [82, 66, 77],
        [84, 64, 76],
        ...,
        [97, 75, 86],
        [92, 75, 89],
        [95, 74, 91]],

       [[82, 62, 75],
        [82, 63, 74],
        [83, 62, 73],
        ...,
        [96, 74, 90],
        [94, 75, 92],
        [95, 75, 94]],

       ...,

       [[25, 40, 47],
        [25, 42, 45],
        [25, 46, 49],
        ...,
        [20, 24, 32],
        [20, 27, 34],
        [22, 28, 36]],

       [[22, 40, 43],
        [24, 43, 45],
        [26, 48, 48],
        ...,
        [21, 25, 32],
        [21, 29, 34],
        [22, 32, 36]],

       [[20, 38, 40],
        [23, 42, 42],
        [26, 45, 47],
        ...,
        [22, 25, 32],
        [23, 28, 34],
        [23, 33, 35]]], dtype=uint8)

In [58]:
np.round_(annotation)

,#item,x,y,dx,dy,label
0,0,273.0,344.0,47.0,47.0,mango
1,1,248.0,356.0,47.0,47.0,mango
2,2,373.0,395.0,47.0,47.0,mango
3,3,391.0,418.0,47.0,47.0,mango
4,4,333.0,382.0,32.0,43.0,mango
5,5,327.0,267.0,29.0,31.0,mango
6,6,345.0,242.0,31.0,41.0,mango


In [59]:
target[:, :4]

array([[364.        , 344.        , 320.15895382, 293.36921537],
       [330.66666667, 356.        , 295.15895382, 302.36921537],
       [497.33333333, 395.        , 420.15895382, 331.61921537],
       [521.33333333, 418.        , 438.15895382, 348.86921537],
       [443.50877193, 381.57894737, 364.63157895, 318.12821053],
       [436.49122807, 267.36842105, 355.88042105, 223.81349179],
       [459.64912281, 242.10526316, 376.1000421 , 212.57417862]])

In [60]:
cv2.imshow('image',img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [61]:
cfg['min_dim']

300

In [62]:
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net

/home/yuvraj/Desktop/work/HM/ssd.pytorch-master/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/yuvraj/Desktop/work/HM/ssd.pytorch-master/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [63]:
if args.cuda:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

In [64]:
if args.resume:
    print('Resuming training, loading {}...'.format(args.resume))
    ssd_net.load_weights(args.resume)
else:
    vgg_weights = torch.load(args.save_folder + args.basenet)
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)

Loading base network...


In [65]:
if args.cuda:
    net = net.cuda()

In [66]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = args.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def xavier(param):
    init.xavier_uniform(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()


def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 3)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )


def update_vis_plot(iteration, loc, conf, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 3)).cpu() * iteration,
        Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )
    # initialize epoch plot on first iteration
    if iteration == 0:
        viz.line(
            X=torch.zeros((1, 3)).cpu(),
            Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu(),
            win=window2,
            update=True
        )

In [67]:
if not args.resume:
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

Initializing weights...


__main__:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [68]:
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum,
                      weight_decay=args.weight_decay)
criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                         False, args.cuda)

In [69]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')

epoch_size = len(dataset) // args.batch_size
print('Training SSD on:', dataset.name)
print('Using the specified args:')
print(args)

step_index = 0

if args.visdom:
    vis_title = 'SSD.PyTorch on ' + dataset.name
    vis_legend = ['Loc Loss', 'Conf Loss', 'Total Loss']
    iter_plot = create_vis_plot('Iteration', 'Loss', vis_title, vis_legend)
    epoch_plot = create_vis_plot('Epoch', 'Loss', vis_title, vis_legend)

data_loader = data.DataLoader(dataset, args.batch_size,
                              num_workers=args.num_workers,
                              shuffle=True, collate_fn=detection_collate,
                              pin_memory=True)


Loading the dataset...
Training SSD on: VOC0712
Using the specified args:
Namespace(basenet='vgg16_reducedfc.pth', batch_size=4, cuda=True, dataset='VOC', dataset_root='/home/yuvraj/Desktop/work/HM/acfr-fruit-dataset/', foo=False, gamma=0.1, lr=0.0005, momentum=0.9, num_workers=4, resume=None, save_folder='weights/', start_iter=0, visdom=False, weight_decay=0.0005)


In [70]:
batch_iterator = iter(data_loader)
for iteration in range(args.start_iter, cfg['max_iter']):
    if args.visdom and iteration != 0 and (iteration % epoch_size == 0):
        update_vis_plot(epoch, loc_loss, conf_loss, epoch_plot, None,
                        'append', epoch_size)
        # reset epoch loss counters
        loc_loss = 0
        conf_loss = 0
        epoch += 1

    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, args.gamma, step_index)

    # load train data
    try:
        images, targets = next(batch_iterator)
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)

    if args.cuda:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    # forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.data[0]
    conf_loss += loss_c.data[0]

    if iteration % 10 == 0:
        print('timer: %.4f sec.' % (t1 - t0))
        print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data[0]), end=' ')

    if args.visdom:
        update_vis_plot(iteration, loss_l.data[0], loss_c.data[0],
                        iter_plot, epoch_plot, 'append')

    if iteration != 0 and iteration % 5000 == 0:
        print('Saving state, iter:', iteration)
        torch.save(ssd_net.state_dict(), 'weights/fruit' +
                   repr(iteration) + '.pth')
torch.save(ssd_net.state_dict(),
           args.save_folder + '' + args.dataset + '.pth')


__main__:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/yuvraj/anaconda3/envs/tf/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
__main__:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
__main__:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
__main__:43: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


timer: 14.9733 sec.
iter 0 || Loss: 17.8809 || timer: 0.6206 sec.
iter 10 || Loss: 8.2514 || timer: 0.6973 sec.
iter 20 || Loss: 14.6094 || timer: 0.5275 sec.
iter 30 || Loss: 12.4495 || timer: 0.5306 sec.
iter 40 || Loss: 24.1649 || timer: 0.5249 sec.
iter 50 || Loss: 3594.5168 || timer: 0.5281 sec.
iter 60 || Loss: nan || 

Process Process-2:
Process Process-3:
Process Process-4:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/site-packages/torch/utils/data/dataloader.py", l

RuntimeError: DataLoader worker (pid 4813) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

In [ ]:
data_loader = data.DataLoader(dataset, args.batch_size,
                              num_workers=args.num_workers,
                              shuffle=True, collate_fn=detection_collate,
                              pin_memory=True)

In [ ]:
batch_iterator = iter(data_loader)

In [ ]:
next(batch_iterator)

In [401]:
images, targets = next(batch_iterator)

IndexError: Traceback (most recent call last):
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/yuvraj/anaconda3/envs/tf/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/yuvraj/Desktop/work/HM/ssd.pytorch-master/data/voc0712.py", line 106, in __getitem__
    im, gt, h, w = self.pull_item(index)
  File "/home/yuvraj/Desktop/work/HM/ssd.pytorch-master/data/voc0712.py", line 125, in pull_item
    img, boxes, labels = self.transform(img, target[:, :4], target[:, 4])
IndexError: too many indices for array


In [402]:
images

tensor([[[[   0.0000,    0.0000,    0.0000,  ...,   97.6198,    5.8867,
           -104.3848],
          [   0.0000,    0.0000,    0.0000,  ...,   88.5270,   29.8855,
            -78.2791],
          [   0.0000,    0.0000,    0.0000,  ...,   88.8228,   79.1799,
            -43.0308],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,  -13.6140,   -5.4604,
              4.2116],
          [   0.0000,    0.0000,    0.0000,  ...,  -30.2417,  -17.0928,
            -10.6018],
          [   0.0000,    0.0000,    0.0000,  ...,  -46.6505,  -35.9037,
            -25.0483]],

         [[   0.0000,    0.0000,    0.0000,  ...,  171.7531,   81.6541,
            -38.6238],
          [   0.0000,    0.0000,    0.0000,  ...,  154.8368,  105.6411,
             -9.2565],
          [   0.0000,    0.0000,    0.0000,  ...,  155.7580,  151.5434,
             27.3409],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,  -43.7821,  -37.7712,
            -27.3819],
          [   0.00

In [403]:
images.shape

torch.Size([4, 3, 300, 300])

In [404]:
targets

[tensor([[0.7530, 0.0371, 0.8459, 0.1556, 1.0000],
         [0.7025, 0.0993, 0.7752, 0.2526, 1.0000],
         [0.7746, 0.1391, 0.8231, 0.2297, 1.0000],
         [0.7385, 0.2275, 0.8153, 0.3390, 1.0000],
         [0.7335, 0.3830, 0.8102, 0.4945, 1.0000],
         [0.7118, 0.5523, 0.7805, 0.6638, 1.0000],
         [0.7501, 0.5622, 0.8188, 0.7643, 1.0000],
         [0.6960, 0.5697, 0.7646, 0.7718, 1.0000]], device='cpu'),
 tensor([[0.4367, 0.2303, 0.5033, 0.3116, 0.0000],
         [0.3882, 0.0428, 0.4481, 0.1160, 0.0000],
         [0.2878, 0.0903, 0.3477, 0.1636, 0.0000]], device='cpu'),
 tensor([[0.0000, 0.5830, 0.0807, 0.6911, 0.0000],
         [0.9114, 0.4881, 0.9738, 0.5456, 0.0000]], device='cpu'),
 tensor([[0.9718, 0.5573, 1.0000, 0.5893, 1.0000],
         [0.7387, 0.7007, 0.7671, 0.7285, 1.0000],
         [0.7688, 0.7052, 0.7949, 0.7373, 1.0000],
         [0.7186, 0.6789, 0.7399, 0.7110, 1.0000],
         [0.8997, 0.4324, 0.9282, 0.4623, 1.0000]], device='cpu')]

In [ ]:
img = cv2.imread(dataset._imgpath % img_id)

In [ ]:
dataset._imgpath

In [ ]:
img_id

In [ ]:
dataset._imgpath % img_id

In [ ]:
len(dataset) // args.batch_size

In [ ]:
471*4